In [1]:
import os
os.chdir('C:\\Users\\HP\\Desktop\\Akaike\\Basics')
os.getcwd()

'C:\\Users\\HP\\Desktop\\Akaike\\Basics'

In [2]:
import pandas as pd
import json
from dotenv import load_dotenv
import re
import copy
import plotly.graph_objects as go
from langchain_community.chat_models import  ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
load_dotenv(".env")


True

## automatic metrics part 

In [3]:
#  def metrics_for_table_prompt( df_head: str, df_shape: str = '', df_nunique: str = '',
#                                  column_description: str = ""):
#         prompt = '''
# Consider yourself as a business Analyst and user as a business stakeholder.
# You will be given, head(1st 5 rows) of the dataset, no. of unique values in the each column.
# You are tasked to do the following:
# - Understand the type of data given and basis that generate useful metrics for the business stakeholder strictly based on given data
# - The metrics you come up with can be of the following categories:
#     - Aggregates: Single number aggreagates based the dataset. For eg: Total Profit, Average order value, etc.
#     - Time Series: In case, data contains some time series, give metrics which highlight trends over time. For Example: category wise sales month on month, average closing value every quarter  etc.
#     - Group Aggregation: In case of categorical data, analysis based on that value can also be useful. For eg: Average order value per Product category, Age Group wise sales Analysis. Profit Across gender
#     - Combination: Combination of Time Series and Group Aggregation
# - Make sure Metrics are useful to the stakeholder and not gibberish
# Example for Reference:
# Head of the dataset:
# 'Order_Date,Time,Aging,Customer_Id,Gender,Device_Type,Customer_Login_type,Product_Category,Product,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Payment_method\n2018-01-02,10:56:33,8.0,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1.0,0.3,46.0,4.6,Medium,credit_card\n2018-07-24,20:41:37,2.0,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1.0,0.3,112.0,11.2,Medium,credit_card\n2018-11-08,08:38:49,8.0,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5.0,0.1,31.2,3.1,Critical,credit_card\n2018-04-18,19:28:06,7.0,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1.0,0.3,26.2,2.6,High,credit_card\n2018-08-13,21:18:39,9.0,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1.0,0.3,160.0,16.0,Critical,credit_card\n'
# shape of the dataset: (51290,16)
# No. of unique values per column:
# 'Order_Date 356\nTime 35275\nAging 11\nCustomer_Id38997\nGender 2\nDevice_Type2\nCustomer_Login_type4\nProduct_Category 4\nProduct 42\nSales 39\nQuantity 5\nDiscount 5\nProfit 391\nShipping_Cost153\nOrder_Priority 4\nPayment_method 5'

# Thought: This is a sales dataset for my stakeholder, metrics related to Average Profit(Aggregates), Average Sales(Aggregates), Average Shipping_Cost(Aggregates),Month on Month Trends of Sales/profit/Shipping cost(Time Series), this can be best visualized through line charts,Sales/Profit/Average order value Across payment method/Device_Type/Order_Priority(Group Aggregation), these can be best visualized through either bar plots of grouped bar plots or pie charts or stacked chart based on number of unique values.Trend of Sales across males and females over time (combination), this can be analyzed threough multiline line charts
# Result :{{"aggregates":["Average Profit", "Average Sales", "Average Shipping_Cost"],"time_series":[{{"metric": "Month on month sales_trend","chart_type":"line",groupby=None}},...],"group_aggregation":[{{"metric":"Revenue per payment method","chart_type":"bar"}}],"combination"[{{"metric":"Sales across gender over time","chart_type":"multiline"}}]}}

# Important Points to Note:
# - All datasets might not have all the 3 types of metrics, leave it as blank list in such cases
# - Maximum possible number of unique values for the column to be considered for group aggregates is 8
# - Chart types given in thought are only for reference, you may give other chart types which could help visualize metric better based on number of unique values and shape of the dataset,like box plot, pie chartetc.
# - possible chart types include ['line', 'bar', 'area', 'histogram', 'pie', 'stacked_bar', 'multiline'(combination of group aggregate and time series), 'grouped_bar']
# - Different metrics of a Metric type can have different visualizations
# - Give metrics name in a professional and formal manner, remember a business stakeholder has to see it.
# - Give atmost 4 metrics for each catgory
# - Use the same structure as given in the above example, JSON keys should exactly be as mentioned
# - Make sure that the output is json serizable, need to directly use json.loads on the output
# Here is what you have:
# Head of the dataset:
# {df_head}
# column description:{column_description}
# shape of the dataset: {df_shape}
# No. of unique values per column:
# {df_nunique}
# Result:
#         '''
#         return prompt.format(df_head=df_head, column_description=column_description, df_shape=df_shape,
#                                      df_nunique=df_nunique)


In [4]:
def metrics_for_table_prompt(df_head: str, df_info: str, df_shape: str = '', df_nunique: str = '', column_description: str = ""):
    prompt = '''
    Consider yourself as a business analyst and the user as a business stakeholder.
    You will be given the head (1st 5 rows) of the dataset, the number of unique values in each column.
    You are tasked to do the following:
    - Understand the type of data given and based on that generate useful metrics for the business stakeholder strictly based on the given data.
    - The metrics you come up with can be of the following categories:
        - Aggregates: Single number aggregates based on the dataset. For example: Total Profit, Average order value, etc.
        - Time Series: In case the data contains some time series, give metrics which highlight trends over time. For example: category-wise sales month on month, average closing value every quarter, etc.
        - Group Aggregation: In case of categorical data, analysis based on that value can also be useful. For example: Average order value per Product category, Age Group wise sales Analysis, Profit Across gender, etc.
        - Combination: Combination of Time Series and Group Aggregation.
    - Make sure Metrics are useful to the stakeholder and not gibberish.
    Example for Reference:
    Head of the dataset:
    'Order_Date,Time,Aging,Customer_Id,Gender,Device_Type,Customer_Login_type,Product_Category,Product,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Payment_method\n2018-01-02,10:56:33,8.0,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1.0,0.3,46.0,4.6,Medium,credit_card\n2018-07-24,20:41:37,2.0,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1.0,0.3,112.0,11.2,Medium,credit_card\n2018-11-08,08:38:49,8.0,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5.0,0.1,31.2,3.1,Critical,credit_card\n2018-04-18,19:28:06,7.0,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1.0,0.3,26.2,2.6,High,credit_card\n2018-08-13,21:18:39,9.0,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1.0,0.3,160.0,16.0,Critical,credit_card\n'
    shape of the dataset: (51290,16)
    No. of unique values per column:
    'Order_Date 356\nTime 35275\nAging 11\nCustomer_Id38997\nGender 2\nDevice_Type2\nCustomer_Login_type4\nProduct_Category 4\nProduct 42\nSales 39\nQuantity 5\nDiscount 5\nProfit 391\nShipping_Cost153\nOrder_Priority 4\nPayment_method 5'
    Info of the dataset:
    Order_Date: 51290 non-null, object\nTime: 51290 non-null, object\n Aging: 51289 non-null, float64\nCustomer_Id: 51290 non-null, int64\nGender: 51290 non-null, object\nDevice_Type: 51290 non-null, object
    \nCustomer_Login_type: 51290 non-null, object\nProduct_Category: 51290 non-null, object\nProduct: 51290 non-null, object\nSales: 51289 non-null, float64\nQuantity: 51288 non-null, float64\n
    Discount: 51289 non-null, float64\nProfit: 51290 non-null, float64\nShipping_Cost: 51289 non-null, float64\nOrder_Priority: 51288 non-null, object\nPayment_method: 51290 non-null, object
    Result:
    {{
    "aggregates": ["Average Profit", "Average Sales", "Average Shipping Cost"],
    "time_series": [{{"metric": "Month-on-Month Sales Trend", "chart_type": "line", "groupby": "Order_Date"}}, {{"metric": "Yearly Shipping Cost Trend", "chart_type": "line", "groupby": "Order_Date"}}],
    "group_aggregation": [{{"metric": "Average Order Value per Product Category", "chart_type": "bar", "groupby": "Product_Category"}}, {{"metric": "Profit Distribution Across Gender", "chart_type": "pie", "groupby": "Gender"}}, {{"metric": "Sales Distribution Across Device Type", "chart_type": "bar", "groupby": "Device_Type"}}],
    "combination": [{{"metric": "Monthly Sales Trend Across Gender", "chart_type": "multiline", "groupby": "Order_Date"}}, {{"metric": "Quarterly Profit Trend Across Product Category", "chart_type": "multiline", "groupby": "Product_Category"}}]
    }}

    Important Points to Note:
    - All datasets might not have all the 3 types of metrics, leave it as a blank list in such cases.
    - Maximum possible number of unique values for the column to be considered for group aggregates is 8.
    - Chart types given in thought are only for reference; you may give other chart types which could help visualize metrics better based on the number of unique values and the shape of the dataset, like box plot, pie chart, etc.
    - Possible chart types include ['line', 'bar', 'area', 'histogram', 'pie', 'stacked_bar', 'multiline' (combination of group aggregate and time series), 'grouped_bar'].
    - Different metrics of a Metric type can have different visualizations.
    - Give metrics names in a professional and formal manner; remember a business stakeholder has to see it.
    - Give at most 4 metrics for each category.
    - Avoid redundancy in metrics; choose the most informative and unique metrics.
    - Use the same structure as given in the above example; JSON keys should be exactly as mentioned.
    - Make sure that the output is JSON serializable; need to directly use json.loads on the output.
    Here is what you have:
    Head of the dataset:
    {df_head}
    Column description: {column_description}
    Shape of the dataset: {df_shape}
    No. of unique values per column:
    {df_nunique}
    Info of the dataset:
    {df_info}
    Result:
    '''
    return prompt.format(df_head=df_head, column_description=column_description, df_shape=df_shape,
                         df_nunique=df_nunique, df_info=df_info)


In [5]:
hmvi = pd.read_csv("../hmvi.csv")

In [6]:
hmvi.head()

,Item,ID Number,Unique ID,Original Balance,Current Balance,Current Contributed Balance,Gross Coupon,Net Coupon,Orig Maturity Term,Rem Maturity Term,...,FICO Score at Loan Origination,Mortgage Insurance Paid By,Mortgage Insurance Percentage,Occupancy (pass-through),Original DTI,Original Months Term,Paid-Through Date,Property Type (pass-through),Loan Purpose (pass-through),Property Sale Price
0,1,1440798,LNRADRE231-1440798,1224000.0,1152000.00,1152000.00,3.550,3.550,361,339.0,...,801.0,BPMI,8.4,Primary,32.0,50,20231031,Single,Other,1440000.0
1,2,1513524,LNRADRE231-1513524,356046.0,346960.11,346960.11,4.680,4.680,361,343.0,...,745.0,BPMI,15.0,Primary,35.0,38,20231031,PUD,Other,374785.0
2,3,1533011,LNRADRE231-1533011,1127312.0,1110300.55,1110300.55,6.375,6.375,361,348.0,...,776.0,BPMI,15.0,Primary,44.0,61,20231031,Single,Other,1304240.0
3,4,1545955,LNRADRE231-1545955,604700.0,589631.13,589631.13,3.350,3.350,361,346.0,...,738.0,BPMI,7.2,Primary,35.0,39,20231031,PUD,Other,671889.0
4,5,1548095,LNRADRE231-1548095,599182.0,583136.26,583136.26,2.875,2.875,361,347.0,...,766.0,BPMI,15.0,Primary,38.0,68,20231031,PUD,Other,665758.0


In [7]:
def model_response(query:str):
    chat = ChatOpenAI(model="gpt-4o",
                    temperature = 0,
                    model_kwargs= {
                    "response_format": {"type": "json_object"},
                    "seed" : 42})

    messages = [
        SystemMessage(
            content="You are a business analyst designed to give key insights,designed to output json"),
        HumanMessage(
            content=query),
    ]
    return json.loads(chat(messages).content)

In [8]:
df_head = hmvi.head().to_csv(index=False)

In [9]:
df_shape = hmvi.shape

In [10]:
df_nunique =hmvi.nunique()

In [11]:
df_info=hmvi.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29000 entries, 0 to 28999
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Item                            29000 non-null  int64  
 1   ID Number                       29000 non-null  int64  
 2   Unique ID                       29000 non-null  object 
 3   Original Balance                29000 non-null  float64
 4   Current Balance                 29000 non-null  float64
 5   Current Contributed Balance     29000 non-null  float64
 6   Gross Coupon                    29000 non-null  float64
 7   Net Coupon                      29000 non-null  float64
 8   Orig Maturity Term              29000 non-null  int64  
 9   Rem Maturity Term               28525 non-null  float64
 10  Orig Amort Term                 29000 non-null  int64  
 11  Rem Amort Term                  28525 non-null  float64
 12  Maturity Date                   

In [12]:
df_nunique = str(df_nunique).replace("  "," ")
df_nunique

'Item               29000\nID Number             29000\nUnique ID             29000\nOriginal Balance          9726\nCurrent Balance          27215\nCurrent Contributed Balance    27215\nGross Coupon            1216\nNet Coupon             1216\nOrig Maturity Term          15\nRem Maturity Term          44\nOrig Amort Term           15\nRem Amort Term            44\nMaturity Date            44\nAge                 17\nIssue Date              17\nState                51\nOrig LTV               20\nLTV                1761\nFactor               7975\nFactor (Cur Bal Wtd)        7975\nProperty Code             6\nOccupancy               3\nPurpose                4\nDocumentation             2\nOriginal Appraisal Value      1661\nTerm                121\nBalance at Deal Issuance     27424\nConforming Balance Flag        2\nDocumentation (pass-through)     2\nFICO Score             228\nFICO Score at Loan Origination   228\nMortgage Insurance Paid By      2\nMortgage Insurance Percentage    

## prompt output

In [13]:
dfmetr = model_response(metrics_for_table_prompt(df_head=df_head,df_info=df_info,df_nunique=df_nunique,df_shape = df_shape))


C:\Users\HP\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
C:\Users\HP\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
dfmetr

{'aggregates': ['Average Original Balance',
  'Average Current Balance',
  'Average Gross Coupon',
  'Average Net Coupon'],
 'time_series': [{'metric': 'Monthly Average Current Balance Trend',
   'chart_type': 'line',
   'groupby': 'Issue Date'},
  {'metric': 'Monthly Average Original Balance Trend',
   'chart_type': 'line',
   'groupby': 'Issue Date'}],
 'group_aggregation': [{'metric': 'Average Original Balance by State',
   'chart_type': 'bar',
   'groupby': 'State'},
  {'metric': 'Average Current Balance by Property Type',
   'chart_type': 'bar',
   'groupby': 'Property Type (pass-through)'},
  {'metric': 'Average Gross Coupon by Occupancy',
   'chart_type': 'bar',
   'groupby': 'Occupancy'},
  {'metric': 'Average Net Coupon by Loan Purpose',
   'chart_type': 'bar',
   'groupby': 'Loan Purpose (pass-through)'}],
 'combination': [{'metric': 'Monthly Average Current Balance Trend by State',
   'chart_type': 'multiline',
   'groupby': 'State'},
  {'metric': 'Monthly Average Original B

## deliverables

- optimize the prompt
- check if the metrics generated are correct or not
- check the chart type
- see how it can be connected wit custom metrics